# Modelo Santi

In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split

In [2]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

# Cargo CSV

In [39]:
test_final = pd.read_csv('data/test_final_100k.csv')
test_final = test_final.drop(columns='id')

In [4]:
postulantes_educacion = pd.read_csv('data_preprocesada/fiuba_1_postulantes_educacion.csv')
postulantes_educacion = postulantes_educacion.drop(columns='Unnamed: 0')
postulantes_genero_y_edad = pd.read_csv('data_preprocesada/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.drop(columns='Unnamed: 0')
vistas = pd.read_csv('data_preprocesada/fiuba_3_vistas.csv')
vistas = vistas.drop(columns='Unnamed: 0')
postulaciones = pd.read_csv('data_preprocesada/fiuba_4_postulaciones.csv')
postulaciones = postulaciones.drop(columns='Unnamed: 0')
avisos_online = pd.read_csv('data_preprocesada/fiuba_5_avisos_online.csv')
avisos_online = avisos_online.drop(columns='Unnamed: 0')
avisos_detalle = pd.read_csv('data_preprocesada/fiuba_6_avisos_detalle.csv')
avisos_detalle = avisos_detalle.drop(columns='Unnamed: 0')

# Funciones

In [5]:
# Fijarse de balancear el sample con 50% de postulados y no postulados
# Recibe las series de idpostulante e idaviso
def generador_de_sample_serie(idaviso, idpostulante, n_sample, ciclos):
    idpostulante = pd.DataFrame(idpostulante)
    idaviso = pd.DataFrame(idaviso)
    idpostulante['tem'] = 1
    idaviso['tem'] = 1
    sample_append = pd.DataFrame(columns=[idpostulante.columns[0],'tem',idaviso.columns[0]])
    for i in range(0,ciclos):
        sample_idpostulante = idpostulante.sample(n=n_sample,random_state=i)
        sample_idaviso = idaviso.sample(n=n_sample,random_state=i)
        sample_merge = sample_idaviso.merge(sample_idpostulante)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop(columns='tem')
    sample_append = sample_append.reset_index(drop=True)
    return sample_append

In [6]:
def borrar_tuplas_test_del_sample(sample,test_final):
    df_return = sample.merge(test_final,how='left',on=['idaviso','idpostulante'],indicator=True)
    df_return = df_return[df_return['_merge'] == 'left_only']
    df_return = df_return.drop(columns='_merge')
    return df_return

In [7]:
def genero_edad(tupla_idaviso_idpostulante,postulantes_genero_y_edad):
    df_return = tupla_idaviso_idpostulante.merge(postulantes_genero_y_edad,on='idpostulante',how='left')
    #df_return = df_return.drop(columns='fechanacimiento')
    return df_return

In [8]:
def educacion(tupla_idaviso_idpostulante):
    df_return = tupla_idaviso_idpostulante.merge(postulantes_educacion,on='idpostulante',how='left')
    return df_return

In [9]:
def area_de_trabajo(tupla_a_predecir):
    df_return = tupla_a_predecir.merge(avisos_detalle,on='idaviso',how='left')
    return df_return

In [10]:
def avisos_visitados(tupla_idaviso_idpostulante):
    df_return = tupla_idaviso_idpostulante.merge(vistas,on=['idpostulante','idaviso'],how='left')
    return df_return

In [11]:
def avisos_online_mark(tupla_idaviso_idpostulante):
    df_return = tupla_idaviso_idpostulante.merge(avisos_online,on=['idaviso'],how='left')
    return df_return

# Head

In [12]:
postulantes_educacion.head()

,idpostulante,Doctorado,Master,Otro,Posgrado,Secundario,Terciario/Técnico,Universitario
0,0z5Dmrd,0,0,0,0,0,0,3
1,0z5JW1r,0,0,3,0,3,3,1
2,0z5VvGv,0,0,0,0,3,0,0
3,0zB01pE,0,0,0,0,0,3,1
4,0zB026d,0,0,0,0,3,3,0


In [13]:
postulantes_genero_y_edad.head()

,idpostulante,edad,0.0,FEM,MASC,NO_DECLARA,tasa_idpostulante
0,eo2p,37,0,0,1,0,0.5
1,1d2B,42,0,0,1,0,0.5
2,EBO0,45,0,1,0,0,0.5
3,a6MKW,44,0,0,1,0,0.5
4,6MWd4,44,0,0,1,0,0.5


In [14]:
vistas.head()

,idaviso,idpostulante,visto
0,820850,DrXDRVP,1
1,1112360,dY98A2D,1
2,1141070,1QPJWqB,1
3,1141070,5maBxez,1
4,1141070,QNEdKBW,1


In [15]:
postulaciones.head()

,idaviso,idpostulante
0,1112257047,NM5M
1,1111920714,NM5M
2,1112346945,NM5M
3,1112345547,NM5M
4,1112237522,5awk


In [16]:
avisos_online.head()

,idaviso,online
0,1112355872,1
1,1112335374,1
2,1112374842,1
3,1111984070,1
4,1111822480,1


In [17]:
avisos_detalle.head()

,idaviso,Administración,Atención al Cliente,Comercio Exterior,Comunicacion,Construcción,Diseño,Diseño Otros,Diseño y Urbanismo,Educación,...,Salud,Secretaria,Secretaria de Tráfico,Seguridad e Higiene,Seguros,Tecnologia/Sistemas,Tecnologia/Sistemas de producción,Ventas,Ventas económico-financiero,tasa_idaviso
0,1000872556,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.5
1,9240880,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0.5
2,1110185164,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.5
3,1110513885,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.5
4,1111034024,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0.5


# ML

In [18]:
#idpostulante = test_final['idpostulante'].drop_duplicates()
#idaviso = test_final['idaviso'].drop_duplicates()

In [19]:
idpostulante = postulantes_genero_y_edad.merge(postulantes_educacion,how='outer',on='idpostulante',indicator=True)
idpostulante = idpostulante[idpostulante['_merge'] == 'both']
idpostulante = idpostulante.drop(columns='_merge')
idpostulante = idpostulante['idpostulante'].drop_duplicates()
idpostulante = idpostulante.append(test_final['idpostulante'])
idpostulante = idpostulante.drop_duplicates()

In [20]:
idaviso = avisos_detalle['idaviso'].drop_duplicates()

In [21]:
sample = generador_de_sample_serie(idaviso,idpostulante,100,100)
del idpostulante
del idaviso

In [22]:
sample = borrar_tuplas_test_del_sample(sample,postulaciones)
sample['y'] = 0
postulaciones['y'] = 1
sample = sample.append(postulaciones.sample(n=1000000,random_state=0))
sample = sample.drop_duplicates(keep=False)
sample.shape

(1999330, 3)

In [23]:
sample = genero_edad(sample,postulantes_genero_y_edad)
#sample['sexo'] = sample['sexo'].fillna('NO_DECLARA')
#sample['edad'] = sample['edad'].astype('int8')
#sample = oneHotEncoding(sample,'sexo')

In [24]:
# deja NaN
sample = educacion(sample)
sample = sample.fillna(0)
#sample.iloc[:,7:] = sample.iloc[:,7:].astype('int8')

In [25]:
sample = area_de_trabajo(sample)
#sample = sample.fillna(0)
#sample.iloc[:,14:] = sample.iloc[:,14:].astype('int8')

In [26]:
sample = avisos_visitados(sample)
sample = sample.fillna(0)
#sample.loc[:,'visto'] = sample.loc[:,'visto'].astype('int8')

In [27]:
# deja NaN
sample = avisos_online_mark(sample)
sample = sample.fillna(0)
#sample.loc[:,'online'] = sample.loc[:,'online'].astype('int8')

del postulantes_educacion
del postulantes_genero_y_edad
del vistas
del postulaciones
del avisos_online
del avisos_detalle

In [28]:
sample = sample.drop(columns=['idaviso','idpostulante'])

In [29]:
sample.head()

,y,edad,0.0,FEM,MASC,NO_DECLARA,tasa_idpostulante,Doctorado,Master,Otro,...,Secretaria de Tráfico,Seguridad e Higiene,Seguros,Tecnologia/Sistemas,Tecnologia/Sistemas de producción,Ventas,Ventas económico-financiero,tasa_idaviso,visto,online
0,0,32,0,1,0,0,0.500000,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0.198198,0.0,0.0
1,0,24,0,0,1,0,0.692308,0.0,0.0,3.0,...,0,0,0,0,0,0,0,0.198198,0.0,0.0
2,0,30,0,1,0,0,0.500000,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0.198198,0.0,0.0
3,0,35,0,0,1,0,3.894737,0.0,0.0,3.0,...,0,0,0,0,0,0,0,0.198198,0.0,0.0
4,0,30,0,0,1,0,0.135593,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0.198198,0.0,0.0


In [30]:
# Divido el set
sample, X_test, sample_y_train, y_test = train_test_split(sample.drop(columns='y'),sample['y'],random_state=0,test_size=0.01)

# Perceptron

In [34]:
from sklearn.metrics import roc_auc_score

In [ ]:
from sklearn.linear_model import Perceptron

In [ ]:
model = Perceptron(penalty=None,n_jobs=-1)
model.fit(sample,sample_y_train)
predict = model.predict(X_test)

In [ ]:
roc_auc_score(y_test,predict)

# KNN

In [ ]:
#from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#model = KNeighborsClassifier(n_neighbors=3)
#model.fit(X_train,y_train)
#predict = model.predict(X_test)

In [ ]:
#roc_auc_score(y_test,predict)

# Random Forest

In [31]:
from sklearn.ensemble import RandomForestClassifier

In [32]:
model = RandomForestClassifier(n_jobs=-1)
model.fit(sample, sample_y_train)
predict = model.predict(X_test)

In [35]:
roc_auc_score(y_test,predict)

0.8635837702486419

# Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
model = GradientBoostingClassifier()
model.fit(sample, sample_y_train)
predict = model.predict(X_test)

In [ ]:
roc_auc_score(y_test,predict)

# ABB

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model = DecisionTreeClassifier()
model.fit(sample, sample_y_train)
predict = model.predict(X_test)

In [ ]:
roc_auc_score(y_test,predict)

# Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
model = GaussianNB()
model.fit(sample, sample_y_train)
predict = model.predict(X_test)

In [ ]:
roc_auc_score(y_test,predict)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
model = MultinomialNB()
model.fit(sample, sample_y_train)
predict = model.predict(X_test)

In [ ]:
roc_auc_score(y_test,predict)

# Test Final

In [40]:
test_final = genero_edad(test_final,postulantes_genero_y_edad)

In [41]:
test_final = educacion(test_final)
test_final = test_final.fillna(0)

In [42]:
test_final = area_de_trabajo(test_final)
test_final = test_final.fillna(0)

In [43]:
test_final = avisos_visitados(test_final)
test_final = test_final.fillna(0)

In [44]:
test_final = avisos_online_mark(test_final)
test_final = test_final.fillna(0)

In [45]:
test_final = test_final.drop(columns=['idaviso','idpostulante'])

In [46]:
test_final.head()

,edad,0.0,FEM,MASC,NO_DECLARA,tasa_idpostulante,Doctorado,Master,Otro,Posgrado,...,Secretaria de Tráfico,Seguridad e Higiene,Seguros,Tecnologia/Sistemas,Tecnologia/Sistemas de producción,Ventas,Ventas económico-financiero,tasa_idaviso,visto,online
0,42,0,1,0,0,0.500000,3.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0.5,0.0,0.0
1,31,0,0,1,0,0.500000,0.0,0.0,3.0,0.0,...,0,0,0,0,0,1,0,0.5,0.0,0.0
2,36,0,1,0,0,0.500000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0.5,0.0,0.0
3,69,0,0,1,0,0.500000,0.0,0.0,0.0,3.0,...,0,0,0,1,0,0,0,0.5,0.0,0.0
4,32,0,1,0,0,0.333333,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0.5,0.0,0.0


In [47]:
predict = model.predict(test_final)

In [48]:
predict = pd.DataFrame(predict,columns=['sepostulo'])

In [49]:
predict.to_csv('prediccion')